In [1]:
%load_ext rpy2.ipython

In [24]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data_60.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(dim(tcdd_data))


[1] 43252   136
[1] 136 198


In [ ]:
%%R
library(dplyr)
library(SummarizedExperiment)

sampling_percentage <- 0.60

tcdd_data_dose0 <- filter(tcdd_data, dose == 0)
tcdd_data_dose30 <- filter(tcdd_data, dose == 30)

min_count <- min(nrow(tcdd_data_dose0), nrow(tcdd_data_dose30)) * sampling_percentage
min_count <- floor(min_count)  

sampled_tcdd_dose0 <- sample_n(tcdd_data_dose0, size = min_count)
sampled_tcdd_dose30 <- sample_n(tcdd_data_dose30, size = min_count)

tcdd_data <- bind_rows(sampled_tcdd_dose0, sampled_tcdd_dose30)

final_result <- final_result[, colData(final_result)$external_id %in% tcdd_data$external_id]
matched_indices <- match(tcdd_data$external_id, colData(final_result)$external_id)
final_result <- final_result[, matched_indices]

print(dim(tcdd_data))
print(dim(final_result))


[1]  80 198
[1] 43252    80


In [ ]:
%%R
dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts030_time_course_60.csv"), row.names=FALSE)


 0 30 
40 40 


In [29]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

 [1] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [9] "male"   "male"   "female" "female" "male"   "male"   "male"   "female"
[17] "male"   "male"   "female" "male"   "male"   "male"   "male"   "female"
[25] "female" "female" "male"   "male"   "male"   "male"   "male"   "male"  
[33] "male"   "female" "male"   "male"   "male"   "male"   "male"   "male"  
[41] "female" "female" "male"   "male"   "male"   "male"   "male"   "female"
[49] "male"   "female" "male"   "female" "male"   "male"   "male"   "male"  
[57] "female" "male"   "male"   "male"   "male"   "male"   "female" "male"  
[65] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "female"
[73] "male"   "male"   "male"   "male"   "male"   "male"   "female" "male"  


In [30]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id030_time_course_60.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender030_time_course_60.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id030_time_course_60.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose030_time_course_60.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec030_time_course_60.Rds",sep=""))
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec030_time_course_60.Rds",sep=""))


In [31]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()
print(tcdd.df)

                           V1       V2      V3      V4      V5      V6       V7
ENSMUSG00000079800.2        0        0       0       0       0       0      100
ENSMUSG00000095092.1        0        0       0       0       0       0        0
ENSMUSG00000079192.2        0        0       0       0       0       0        0
ENSMUSG00000079794.2        0        0       0       0       0       0        0
ENSMUSG00000094799.1        0        0       0       0       0       0        0
ENSMUSG00000095250.1        0        0       0       0       0       0        0
ENSMUSG00000095787.1        0        0       0       0       0       0        0
ENSMUSG00000096100.1        0        0       0       0       0       0      200
ENSMUSG00000094054.1        0      250       0       0       0       0        0
ENSMUSG00000095672.1        0        0       0       0       0       0        0
ENSMUSG00000079190.3        0        0       0      50       0       0        0
ENSMUSG00000094514.1        0        0  

ENSMUSG00000051285.17   46861    47650   31767   35818   35161   21685    52879
ENSMUSG00000048538.7   143348   311451  241479  134230  215912   64953   292868
ENSMUSG00000103709.1        0        0       0       0       0       0        0
ENSMUSG00000102768.1        0        0       0       0       0       0        0
ENSMUSG00000097797.6        0        0       0       0       0      25       50
ENSMUSG00000103067.1        0        0       0       0       0       0        0
ENSMUSG00000104226.1      100      150       0      86      50       0      150
ENSMUSG00000103903.1        0       50       0       0       0       0        0
ENSMUSG00000102647.1        0        0       0       0       0       0        0
ENSMUSG00000086235.1        0      200     300      50     399       0      200
ENSMUSG00000102253.1        0        0       0       0       0       0        0
ENSMUSG00000103884.1        0        0       0       0       0       0        0
ENSMUSG00000084353.1     1355     2747  

ENSMUSG00000101652.1      100       50      50     100       0     100       50
ENSMUSG00000100814.1        0        0       0       0       0       0        0
ENSMUSG00000025925.14    6278     6091    4311    4053    4250    2546     4116
ENSMUSG00000104379.1       96       50      50     250     150     250      198
ENSMUSG00000088943.1        0        0       0       0       0       0        0
ENSMUSG00000032719.4       50       50      50     200     100       0      500
ENSMUSG00000100652.1       45      490      47      97       0       0      191
ENSMUSG00000067795.13       0        0       0       0       0       0        0
ENSMUSG00000043716.13   42122    42590   42229   31757   31903   15758    44140
ENSMUSG00000025921.7    12410    16624   12817   11119    9576    5302    14158
ENSMUSG00000104165.1        0        0       0       0       0       0        0
ENSMUSG00000100868.1        0        0       0       0       0      50        0
ENSMUSG00000089982.7        0        0  

ENSMUSG00000026141.13       0        0       0       0       0       0        0
ENSMUSG00000099997.1     3643     5236    4645    2638    4602    1643     6244
ENSMUSG00000100131.1      238      100      37      50     100      49      445
ENSMUSG00000067736.2       72        0       5       0       0       0        0
ENSMUSG00000101939.1    63044    78854   65892   35242   48972   19637    50374
ENSMUSG00000101111.1  7024369  8189415 6984547 5937886 6393457 2778646  8767170
ENSMUSG00000100862.1  8859479 10223519 8285126 7075012 8005408 3594813 11122322
ENSMUSG00000102070.1  5354949  6421299 5800987 4407645 5241109 2216911  7073364
ENSMUSG00000101249.1  7531493  9092622 8239537 7006297 6729982 3380439  9761085
ENSMUSG00000073725.8    33114    35427   21915   29239   23067   13706    35716
ENSMUSG00000100808.1        0        0       0       0       0       0        0
ENSMUSG00000100450.1        0        0       0       0       0       0      150
ENSMUSG00000080849.1        0        0  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ENSMUSG00000099472.1         0       0       50        0        0        0
ENSMUSG00000033227.7        50       0      199        0        0       50
ENSMUSG00000026167.14     1296   18224    62999    17645    18780    12163
ENSMUSG00000090071.4         0       0        0        0        0        0
ENSMUSG00000089964.1         0       0        0        0        0        0
ENSMUSG00000055197.4         0       0        0        0        0        0
ENSMUSG00000006546.3         0      44        0        0       50        0
ENSMUSG00000089653.1         0       0        0        0        0        0
ENSMUSG00000047021.14        0       0       50        0        0        0
ENSMUSG00000006538.12      866     595     1049      200      660      846
ENSMUSG00000103145.1         0       0        0        0        0        0
ENSMUSG00000026162.7       738     893     3869     2168     1312      711
ENSMUSG00000026205.8        50     100      185        0        0       50
ENSMUSG00000033159.14    

ENSMUSG00000097564.2      1140     793      752      964      855     1547
ENSMUSG00000101450.1         0      50        0      200      315       50
ENSMUSG00000026150.14    11697   25684    33436    35853    40291    26387
ENSMUSG00000026149.2         0      50      100        0      250      250
ENSMUSG00000114299.1         0       0        0      144        0        0
ENSMUSG00000099771.1         0       0        0        0        0        0
ENSMUSG00000102280.1         0       0        0        0        0        0
ENSMUSG00000100170.1        48       0        0        0        0        0
ENSMUSG00000100744.1       595     499     1240      299      248      646
ENSMUSG00000093782.1         0       0        0        0        0        0
ENSMUSG00000026159.13     7614   16330    10550    23381    33101    17235
ENSMUSG00000113880.1        99     250       60        0      476      100
ENSMUSG00000113973.1         0       0        0        0        0        0
ENSMUSG00000113846.1     

ENSMUSG00000103609.1         0       0        0        0        0       50
ENSMUSG00000026241.5         0       0        0        0        0        0
ENSMUSG00000100225.1         0       0        0        0        0        0
ENSMUSG00000100768.1         0       0        0        0        0        0
                           V58      V59      V60      V61      V62     V63
ENSMUSG00000079800.2         0        0        0        0        0       0
ENSMUSG00000095092.1         0        0        0        0        0       0
ENSMUSG00000079192.2         0        0        0        0        0       0
ENSMUSG00000079794.2        50        0        0        0      100     100
ENSMUSG00000094799.1         0        0        0        0        0       0
ENSMUSG00000095250.1         0        0        0      100        0       0
ENSMUSG00000095787.1         0        0        0        0        0       0
ENSMUSG00000096100.1         0       29        0        0        0       0
ENSMUSG00000094054.1     

ENSMUSG00000102768.1         0        0        0        0        0       0
ENSMUSG00000097797.6       224      164      350       84      200       0
ENSMUSG00000103067.1         0        0        0        0       48      48
ENSMUSG00000104226.1       300      300      150      598      436     136
ENSMUSG00000103903.1         0       50        0        0      100       0
ENSMUSG00000102647.1         0        0        0        0       50       0
ENSMUSG00000086235.1       350      300      350      200      300     100
ENSMUSG00000102253.1         0        0        0        0        0       0
ENSMUSG00000103884.1         0        0        0        0        0       0
ENSMUSG00000084353.1      3012     3366     2506     2523     3743    1165
ENSMUSG00000103933.1     28265    22287    30659    27454    33627   14987
ENSMUSG00000076135.1         0        0        0        0        0       0
ENSMUSG00000086195.1        31       31       78       53       67      30
ENSMUSG00000104504.1     

ENSMUSG00000104165.1         0        0        0        0        0       0
ENSMUSG00000100868.1         0        0        0        0       50      49
ENSMUSG00000089982.7         0        0        0        0        0       0
ENSMUSG00000025920.19     6794     6467     4476     7109     6756    2407
ENSMUSG00000025939.18    15273    11956    11130    12079    17851    6445
ENSMUSG00000102457.1        50        0       50        0      100       0
ENSMUSG00000079658.9     24574    23290    17975    19925    28178    8993
ENSMUSG00000097744.1       148      100      114       49      130       0
ENSMUSG00000100959.1       150      105      100       50        0       0
ENSMUSG00000025940.6     37072    38268    26715    45528    38189    8705
ENSMUSG00000025779.10     3687     4256     2880     2820     4883    1892
ENSMUSG00000100093.1         0        0        0        0      100      49
ENSMUSG00000091020.3      4287     3096     3394     3788     5966    1194
ENSMUSG00000101640.1     

In [32]:
%%R
colnames(tcdd.df) <- colData(final_result)$external_id
print(tcdd.df)

          0          0
ENSMUSG00000102321.1           0          0          0          0          0
ENSMUSG00000091318.2           0          0          0          0          0
ENSMUSG00000026058.11          0          0          0          0          0
ENSMUSG00000102926.1           0          0          0          0          0
ENSMUSG00000100481.1        3637       3306       2637       3322       2579
ENSMUSG00000100590.1         250        150          0         50          0
ENSMUSG00000101612.2         100        195        199         49        196
ENSMUSG00000101682.1         647        647        546        300        639
ENSMUSG00000102714.1           0          0          0          0          0
ENSMUSG00000102093.1           0          0          0          0          0
ENSMUSG00000026134.11       1645        399       1276       1512       1326
ENSMUSG00000103746.1           0          0         50         36          0
ENSMUSG00000102198.1           0          0          

ENSMUSG00000045515.3           0          0          0          0          0
ENSMUSG00000077448.1           0          0          0          0          0
ENSMUSG00000060679.14      11914      10083       8400      11555       8598
ENSMUSG00000041907.9           0          0          0          0          0
ENSMUSG00000100269.1           0          0          0         23          0
ENSMUSG00000041878.3         146         35        240        300        150
ENSMUSG00000070939.9       25001      33164      22325      19843      18733
ENSMUSG00000010290.7        6251       6686       5309       5305       7034
ENSMUSG00000094497.1           0          0          0          0         46
ENSMUSG00000008136.14          0        200        100         50        134
ENSMUSG00000099694.1           0        100          0         50        150
ENSMUSG00000099576.1           0          0          0          0          0
ENSMUSG00000101923.1         150        200          0        350        150

ENSMUSG00000072324.4         899       1798        843        950       1497
ENSMUSG00000078291.1          50        150        100         50          0
ENSMUSG00000062939.11          0        380          0        200        164
ENSMUSG00000109599.1         400        294        350        350        445
ENSMUSG00000099569.1           0          0          0          0          0
ENSMUSG00000101628.1       20608      19473      13706      15005      11717
ENSMUSG00000026104.14      34182      35969      23762      24838      20405
ENSMUSG00000026103.14       2241       3086       1841       2256       1626
ENSMUSG00000002881.14      31130      30248      23264      28946      22043
ENSMUSG00000097833.1         551        198         50          0          0
ENSMUSG00000101501.1        3998       4249       2998       3199       3000
ENSMUSG00000099878.1           0          0          0          0          0
ENSMUSG00000100120.1           0          0          0          0         50

ENSMUSG00000026032.8       18499      25339      19797      13067      21266
ENSMUSG00000100202.1           0          0          0          0          0
ENSMUSG00000059647.5        3494       2237       2150       1743       1699
ENSMUSG00000102059.1         760       2373       2065       1241       1519
ENSMUSG00000047528.13       2927       1541       1548       1052       1033
ENSMUSG00000026031.15      38822      65923      43640      31608      32777
ENSMUSG00000100236.1          50          0          0          0          0
ENSMUSG00000102719.1           0         50         94          0        200
ENSMUSG00000026029.14      13462      13972      12069      10054      11691
ENSMUSG00000099719.1           0          0          0          0          0
ENSMUSG00000026028.12      10991      13176      10049       8870       8387
ENSMUSG00000026027.13      24545      21152      15487      22036      16672
ENSMUSG00000047383.8          50         50          0          0          0

ENSMUSG00000105060.1           0         18          0          0          0
ENSMUSG00000064612.2          14         72         10          0          0
ENSMUSG00000064602.1          92          0        125        196        258
ENSMUSG00000025967.16      42374      53069      47565      39323      47084
ENSMUSG00000046856.13          0          0          0          0          0
ENSMUSG00000027520.15          0          0          0          0          0
ENSMUSG00000081643.1        5042       6286       4887       4193       4546
ENSMUSG00000087823.1           0          0          0          0          0
ENSMUSG00000025964.15       2380       7077       3284       2048       3607
ENSMUSG00000115378.1           0          0          0          0          0
ENSMUSG00000083066.1           0          0          0          0          0
ENSMUSG00000069085.5           0          0          0          0          0
ENSMUSG00000025963.5           0         50          0          0          0

...

ENSMUSG00000099946.1           0          0          0         49          0
ENSMUSG00000070896.2          97          0          0        437         98
ENSMUSG00000090682.4        1850       2741        649        700       1400
ENSMUSG00000044457.8           0          0          0          0          0
ENSMUSG00000099432.1           0          0          0          0         50
ENSMUSG00000041144.11         50         50        200         50        249
ENSMUSG00000101262.1       21211       6288       4526      18169       5622
ENSMUSG00000101337.6           0          0        126        100          0
ENSMUSG00000100213.1         300        400        454       1148       1023
ENSMUSG00000103129.1           0          0          0          0          0
ENSMUSG00000025986.6       12416       5392       3397       7464       7941
ENSMUSG00000100160.1        1201        781        611        901       1251
ENSMUSG00000101447.1           0          0          0          0          0

ENSMUSG00000025978.14       1304        843        556       1529       1283
ENSMUSG00000097649.2         250        200          0        169        350
ENSMUSG00000046994.9        8713       4365       5390      18659       6340
ENSMUSG00000100215.1        2663       1622       1701       5629       3680
ENSMUSG00000025977.15         50          0          0          0         50
ENSMUSG00000101945.1         540        342        148        395        445
ENSMUSG00000101085.1           0          0          0          0          0
ENSMUSG00000100309.1         947       1199       1900       2245       1844
ENSMUSG00000102145.1        1444       1497       2047       2788       2236
ENSMUSG00000102831.1           0          0          0          0          0
ENSMUSG00000038349.10        800        500        350       1247        642
ENSMUSG00000099950.1           0          0          0          0          0
ENSMUSG00000100944.1           0          0          0          0          0

ENSMUSG00000086795.1           0          0          0          0          0
ENSMUSG00000083118.1           0          0          0          0          0
ENSMUSG00000085965.1           0          0          0          0          0
ENSMUSG00000083355.1         248        190        150        250        188
ENSMUSG00000102549.1         100          0          0         50          0
ENSMUSG00000081382.1       19502      13122      14446      37632      18143
ENSMUSG00000026011.13         50          0          0        150         50
ENSMUSG00000080800.1           0          0         50         50         50
ENSMUSG00000102561.1           0          0          0         50          0
ENSMUSG00000026009.14        550        150        696       1548        446
ENSMUSG00000082804.3           0        100          0         50        150
ENSMUSG00000080785.1           0          0          0          0          0
ENSMUSG00000085248.2           0          0          0          0          0

ENSMUSG00000102374.1         400        300        300        849        500
ENSMUSG00000102776.1         600        434        100       1488        446
ENSMUSG00000082057.1           4         50          0         50        100
ENSMUSG00000103706.1          50        200         50        300        300
ENSMUSG00000103756.1         449        100        200        612        200
ENSMUSG00000103382.1           0        250        150        748        200
ENSMUSG00000026004.15       5983       5794       5619      24587       8812
ENSMUSG00000103553.1         200        100        100        299        100
ENSMUSG00000082769.3        7234       3940       7824       6381       6408
ENSMUSG00000089420.2           0          0          0          0          0
ENSMUSG00000026003.5      162859     127672     117493     324297     191150
ENSMUSG00000077328.1           0          0          0          0          0
ENSMUSG00000061816.15          0          0          0          0          0

ENSMUSG00000026174.9       18926      12432      11861      31128      18831
ENSMUSG00000033364.13       7641       7129       6307      17258       8707
ENSMUSG00000026173.15       2225       2830       2063       3150       2592
ENSMUSG00000093155.1           0          0          0          0          0
ENSMUSG00000026135.17      10114       7857       7731      11111       7826
ENSMUSG00000026172.12      11628       3284       9401      18282       6069
ENSMUSG00000026171.12       9685       9199      10611      21499      13598
ENSMUSG00000033276.18       1366       1286        147        756       1541
ENSMUSG00000033257.14      31058      28210      18513      51209      26027
ENSMUSG00000026170.6      194226     336821     160725     404562     193296
ENSMUSG00000006542.13          0        922         90        567        409
ENSMUSG00000099472.1           0          0          0          0         48
ENSMUSG00000033227.7          98        199         50          0        108

In [33]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [34]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(length(deleted_ensembl_wo_ids))

In [35]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [36]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

 [3859] "ENSMUSG00000088406" "ENSMUSG00000117366" "ENSMUSG00000117459"
 [3862] "ENSMUSG00000117347" "ENSMUSG00000117495" "ENSMUSG00000089189"
 [3865] "ENSMUSG00000098621" "ENSMUSG00000101756" "ENSMUSG00000084750"
 [3868] "ENSMUSG00000092317" "ENSMUSG00000092555" "ENSMUSG00000044538"
 [3871] "ENSMUSG00000048231" "ENSMUSG00000092493" "ENSMUSG00000092296"
 [3874] "ENSMUSG00000092606" "ENSMUSG00000037334" "ENSMUSG00000092377"
 [3877] "ENSMUSG00000092343" "ENSMUSG00000092435" "ENSMUSG00000092385"
 [3880] "ENSMUSG00000088819" "ENSMUSG00000092339" "ENSMUSG00000096477"
 [3883] "ENSMUSG00000091601" "ENSMUSG00000092542" "ENSMUSG00000094884"
 [3886] "ENSMUSG00000092396" "ENSMUSG00000092568" "ENSMUSG00000092477"
 [3889] "ENSMUSG00000062629" "ENSMUSG00000109188" "ENSMUSG00000095286"
 [3892] "ENSMUSG00000059964" "ENSMUSG00000063994" "ENSMUSG00000060017"
 [3895] "ENSMUSG00000083947" "ENSMUSG00000109185" "ENSMUSG00000045202"
 [3898] "ENSMUSG00000044985" "ENSMUSG00000050613" "ENSMUSG00000109212"
 [3901

 [6430] "ENSMUSG00000105310" "ENSMUSG00000104886" "ENSMUSG00000106177"
 [6433] "ENSMUSG00000099854" "ENSMUSG00000091827" "ENSMUSG00000090750"
 [6436] "ENSMUSG00000058643" "ENSMUSG00000105930" "ENSMUSG00000089364"
 [6439] "ENSMUSG00000088668" "ENSMUSG00000096882" "ENSMUSG00000077171"
 [6442] "ENSMUSG00000087768" "ENSMUSG00000105036" "ENSMUSG00000065660"
 [6445] "ENSMUSG00000104866" "ENSMUSG00000098903" "ENSMUSG00000105254"
 [6448] "ENSMUSG00000065329" "ENSMUSG00000105713" "ENSMUSG00000086729"
 [6451] "ENSMUSG00000088842" "ENSMUSG00000105744" "ENSMUSG00000029044"
 [6454] "ENSMUSG00000105491" "ENSMUSG00000104733" "ENSMUSG00000081970"
 [6457] "ENSMUSG00000086505" "ENSMUSG00000093875" "ENSMUSG00000088150"
 [6460] "ENSMUSG00000086815" "ENSMUSG00000088327" "ENSMUSG00000096091"
 [6463] "ENSMUSG00000107220" "ENSMUSG00000106835" "ENSMUSG00000107046"
 [6466] "ENSMUSG00000107248" "ENSMUSG00000080556" "ENSMUSG00000082692"
 [6469] "ENSMUSG00000098884" "ENSMUSG00000115557" "ENSMUSG00000087487"
 [6472

 [6775] "ENSMUSG00000105490" "ENSMUSG00000105575" "ENSMUSG00000105308"
 [6778] "ENSMUSG00000104778" "ENSMUSG00000087104" "ENSMUSG00000085328"
 [6781] "ENSMUSG00000105857" "ENSMUSG00000104980" "ENSMUSG00000105483"
 [6784] "ENSMUSG00000104681" "ENSMUSG00000106729" "ENSMUSG00000099255"
 [6787] "ENSMUSG00000105584" "ENSMUSG00000105726" "ENSMUSG00000105773"
 [6790] "ENSMUSG00000106516" "ENSMUSG00000106395" "ENSMUSG00000106450"
 [6793] "ENSMUSG00000105598" "ENSMUSG00000103500" "ENSMUSG00000088123"
 [6796] "ENSMUSG00000106097" "ENSMUSG00000105227" "ENSMUSG00000106800"
 [6799] "ENSMUSG00000106323" "ENSMUSG00000092402" "ENSMUSG00000098131"
 [6802] "ENSMUSG00000079174" "ENSMUSG00000106466" "ENSMUSG00000118522"
 [6805] "ENSMUSG00000112194" "ENSMUSG00000103298" "ENSMUSG00000104683"
 [6808] "ENSMUSG00000093344" "ENSMUSG00000105344" "ENSMUSG00000089164"
 [6811] "ENSMUSG00000085540" "ENSMUSG00000105177" "ENSMUSG00000061707"
 [6814] "ENSMUSG00000106393" "ENSMUSG00000029620" "ENSMUSG00000118480"
 [6817

 [7120] "ENSMUSG00000104931" "ENSMUSG00000104883" "ENSMUSG00000106401"
 [7123] "ENSMUSG00000104744" "ENSMUSG00000089157" "ENSMUSG00000087917"
 [7126] "ENSMUSG00000104742" "ENSMUSG00000088245" "ENSMUSG00000106098"
 [7129] "ENSMUSG00000106480" "ENSMUSG00000104532" "ENSMUSG00000104934"
 [7132] "ENSMUSG00000105306" "ENSMUSG00000105007" "ENSMUSG00000105463"
 [7135] "ENSMUSG00000104722" "ENSMUSG00000105684" "ENSMUSG00000076530"
 [7138] "ENSMUSG00000104832" "ENSMUSG00000106187" "ENSMUSG00000105319"
 [7141] "ENSMUSG00000077325" "ENSMUSG00000105817" "ENSMUSG00000090781"
 [7144] "ENSMUSG00000106381" "ENSMUSG00000104995" "ENSMUSG00000104849"
 [7147] "ENSMUSG00000105267" "ENSMUSG00000079544" "ENSMUSG00000105539"
 [7150] "ENSMUSG00000105313" "ENSMUSG00000104609" "ENSMUSG00000104922"
 [7153] "ENSMUSG00000105989" "ENSMUSG00000105968" "ENSMUSG00000104697"
 [7156] "ENSMUSG00000088675" "ENSMUSG00000106222" "ENSMUSG00000105110"
 [7159] "ENSMUSG00000105323" "ENSMUSG00000106216" "ENSMUSG00000104551"
 [7162

 [7465] "ENSMUSG00000065882" "ENSMUSG00000107613" "ENSMUSG00000088239"
 [7468] "ENSMUSG00000107839" "ENSMUSG00000107398" "ENSMUSG00000077241"
 [7471] "ENSMUSG00000087850" "ENSMUSG00000088867" "ENSMUSG00000097419"
 [7474] "ENSMUSG00000086643" "ENSMUSG00000087852" "ENSMUSG00000088362"
 [7477] "ENSMUSG00000088781" "ENSMUSG00000107964" "ENSMUSG00000086021"
 [7480] "ENSMUSG00000086019" "ENSMUSG00000086777" "ENSMUSG00000065269"
 [7483] "ENSMUSG00000080536" "ENSMUSG00000083351" "ENSMUSG00000107523"
 [7486] "ENSMUSG00000108044" "ENSMUSG00000118413" "ENSMUSG00000107815"
 [7489] "ENSMUSG00000107862" "ENSMUSG00000108758" "ENSMUSG00000095674"
 [7492] "ENSMUSG00000108644" "ENSMUSG00000108765" "ENSMUSG00000075719"
 [7495] "ENSMUSG00000083309" "ENSMUSG00000108743" "ENSMUSG00000108667"
 [7498] "ENSMUSG00000096075" "ENSMUSG00000095246" "ENSMUSG00000091874"
 [7501] "ENSMUSG00000091652" "ENSMUSG00000075906" "ENSMUSG00000109155"
 [7504] "ENSMUSG00000108904" "ENSMUSG00000109577" "ENSMUSG00000092336"
 [7507

...

[10570] "ENSMUSG00000084421" "ENSMUSG00000080250" "ENSMUSG00000084191"
[10573] "ENSMUSG00000083970" "ENSMUSG00000080046" "ENSMUSG00000083809"
[10576] "ENSMUSG00000094643" "ENSMUSG00000072930" "ENSMUSG00000072929"
[10579] "ENSMUSG00000081533" "ENSMUSG00000084001" "ENSMUSG00000082856"
[10582] "ENSMUSG00000084112" "ENSMUSG00000080071" "ENSMUSG00000082995"
[10585] "ENSMUSG00000083256" "ENSMUSG00000095994" "ENSMUSG00000083778"
[10588] "ENSMUSG00000094885" "ENSMUSG00000096213" "ENSMUSG00000096179"
[10591] "ENSMUSG00000094941" "ENSMUSG00000095820" "ENSMUSG00000093851"
[10594] "ENSMUSG00000096008" "ENSMUSG00000095621" "ENSMUSG00000096338"
[10597] "ENSMUSG00000094964" "ENSMUSG00000082937" "ENSMUSG00000084395"
[10600] "ENSMUSG00000095082" "ENSMUSG00000095534" "ENSMUSG00000081827"
[10603] "ENSMUSG00000098311" "ENSMUSG00000087149" "ENSMUSG00000086286"
[10606] "ENSMUSG00000064524" "ENSMUSG00000070219" "ENSMUSG00000083719"
[10609] "ENSMUSG00000084637" "ENSMUSG00000091863" "ENSMUSG00000088383"
[10612

[10915] "ENSMUSG00000104127" "ENSMUSG00000103872" "ENSMUSG00000102787"
[10918] "ENSMUSG00000104499" "ENSMUSG00000103431" "ENSMUSG00000104191"
[10921] "ENSMUSG00000102492" "ENSMUSG00000104260" "ENSMUSG00000102251"
[10924] "ENSMUSG00000102366" "ENSMUSG00000103909" "ENSMUSG00000102835"
[10927] "ENSMUSG00000103943" "ENSMUSG00000102624" "ENSMUSG00000102138"
[10930] "ENSMUSG00000102582" "ENSMUSG00000103666" "ENSMUSG00000104207"
[10933] "ENSMUSG00000102392" "ENSMUSG00000095172" "ENSMUSG00000102754"
[10936] "ENSMUSG00000102387" "ENSMUSG00000102208" "ENSMUSG00000102265"
[10939] "ENSMUSG00000104490" "ENSMUSG00000102288" "ENSMUSG00000103417"
[10942] "ENSMUSG00000094829" "ENSMUSG00000095520" "ENSMUSG00000103602"
[10945] "ENSMUSG00000102586" "ENSMUSG00000102170" "ENSMUSG00000102421"
[10948] "ENSMUSG00000104032" "ENSMUSG00000102400" "ENSMUSG00000102520"
[10951] "ENSMUSG00000094161" "ENSMUSG00000102726" "ENSMUSG00000102739"
[10954] "ENSMUSG00000103757" "ENSMUSG00000096819" "ENSMUSG00000103994"
[10957

[11260] "ENSMUSG00000100244" "ENSMUSG00000099799" "ENSMUSG00000099999"
[11263] "ENSMUSG00000101806" "ENSMUSG00000101632" "ENSMUSG00000099525"
[11266] "ENSMUSG00000100749" "ENSMUSG00000099963" "ENSMUSG00000100299"
[11269] "ENSMUSG00000101127" "ENSMUSG00000096650" "ENSMUSG00000102058"
[11272] "ENSMUSG00000100034" "ENSMUSG00000099589" "ENSMUSG00000100263"
[11275] "ENSMUSG00000101575" "ENSMUSG00000100785" "ENSMUSG00000099822"
[11278] "ENSMUSG00000101781" "ENSMUSG00000101271" "ENSMUSG00000099504"
[11281] "ENSMUSG00000101377" "ENSMUSG00000101709" "ENSMUSG00000099814"
[11284] "ENSMUSG00000100316" "ENSMUSG00000101284" "ENSMUSG00000100651"
[11287] "ENSMUSG00000101933" "ENSMUSG00000100647" "ENSMUSG00000101815"
[11290] "ENSMUSG00000101538" "ENSMUSG00000100983" "ENSMUSG00000101820"
[11293] "ENSMUSG00000100283" "ENSMUSG00000099832" "ENSMUSG00000101246"
[11296] "ENSMUSG00000101382" "ENSMUSG00000099967" "ENSMUSG00000101161"
[11299] "ENSMUSG00000100705" "ENSMUSG00000101157" "ENSMUSG00000099712"
[11302

[11605] "ENSMUSG00000101350" "ENSMUSG00000100614" "ENSMUSG00000100501"
[11608] "ENSMUSG00000100083" "ENSMUSG00000101991" "ENSMUSG00000100076"
[11611] "ENSMUSG00000101100" "ENSMUSG00000101744" "ENSMUSG00000095807"
[11614] "ENSMUSG00000101317" "ENSMUSG00000101244" "ENSMUSG00000096626"
[11617] "ENSMUSG00000100085" "ENSMUSG00000101424" "ENSMUSG00000100264"
[11620] "ENSMUSG00000100715" "ENSMUSG00000100582" "ENSMUSG00000099551"
[11623] "ENSMUSG00000096347" "ENSMUSG00000101531" "ENSMUSG00000100425"
[11626] "ENSMUSG00000101005" "ENSMUSG00000101300" "ENSMUSG00000101426"
[11629] "ENSMUSG00000101891" "ENSMUSG00000100238" "ENSMUSG00000101103"
[11632] "ENSMUSG00000099866" "ENSMUSG00000101110" "ENSMUSG00000095141"
[11635] "ENSMUSG00000101910" "ENSMUSG00000100579" "ENSMUSG00000094217"
[11638] "ENSMUSG00000100057" "ENSMUSG00000101592" "ENSMUSG00000100609"
[11641] "ENSMUSG00000101993" "ENSMUSG00000099935" "ENSMUSG00000100667"
[11644] "ENSMUSG00000100012" "ENSMUSG00000099836" "ENSMUSG00000100575"
[11647

[11950] "ENSMUSG00000100433" "ENSMUSG00000101060" "ENSMUSG00000102111"
[11953] "ENSMUSG00000099792" "ENSMUSG00000099497" "ENSMUSG00000099479"
[11956] "ENSMUSG00000100849" "ENSMUSG00000101115" "ENSMUSG00000101194"
[11959] "ENSMUSG00000099742" "ENSMUSG00000100999" "ENSMUSG00000101562"
[11962] "ENSMUSG00000100328" "ENSMUSG00000101321" "ENSMUSG00000101098"
[11965] "ENSMUSG00000094739" "ENSMUSG00000102071" "ENSMUSG00000100741"
[11968] "ENSMUSG00000101804" "ENSMUSG00000099621" "ENSMUSG00000095867"
[11971] "ENSMUSG00000100784" "ENSMUSG00000101044" "ENSMUSG00000099670"
[11974] "ENSMUSG00000099636" "ENSMUSG00000094746" "ENSMUSG00000100494"
[11977] "ENSMUSG00000101118" "ENSMUSG00000101748" "ENSMUSG00000101259"
[11980] "ENSMUSG00000100610" "ENSMUSG00000101197" "ENSMUSG00000102016"
[11983] "ENSMUSG00000101340" "ENSMUSG00000099972" "ENSMUSG00000101733"
[11986] "ENSMUSG00000102019" "ENSMUSG00000101465" "ENSMUSG00000095578"
[11989] "ENSMUSG00000099734" "ENSMUSG00000100723" "ENSMUSG00000101981"
[11992

In [37]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

   [1] "ENSMUSG00000000001" "ENSMUSG00000000028" "ENSMUSG00000000049"
   [4] "ENSMUSG00000000058" "ENSMUSG00000000085" "ENSMUSG00000000088"
   [7] "ENSMUSG00000000120" "ENSMUSG00000000126" "ENSMUSG00000000127"
  [10] "ENSMUSG00000000142" "ENSMUSG00000000149" "ENSMUSG00000000154"
  [13] "ENSMUSG00000000168" "ENSMUSG00000000171" "ENSMUSG00000000182"
  [16] "ENSMUSG00000000183" "ENSMUSG00000000184" "ENSMUSG00000000197"
  [19] "ENSMUSG00000000214" "ENSMUSG00000000215" "ENSMUSG00000000216"
  [22] "ENSMUSG00000000223" "ENSMUSG00000000247" "ENSMUSG00000000253"
  [25] "ENSMUSG00000000263" "ENSMUSG00000000275" "ENSMUSG00000000276"
  [28] "ENSMUSG00000000290" "ENSMUSG00000000296" "ENSMUSG00000000301"
  [31] "ENSMUSG00000000303" "ENSMUSG00000000305" "ENSMUSG00000000308"
  [34] "ENSMUSG00000000320" "ENSMUSG00000000326" "ENSMUSG00000000340"
  [37] "ENSMUSG00000000374" "ENSMUSG00000000394" "ENSMUSG00000000399"
  [40] "ENSMUSG00000000409" "ENSMUSG00000000420" "ENSMUSG00000000435"
  [43] "ENSMUSG00000

 [352] "ENSMUSG00000003541" "ENSMUSG00000003546" "ENSMUSG00000003549"
 [355] "ENSMUSG00000003555" "ENSMUSG00000003559" "ENSMUSG00000003573"
 [358] "ENSMUSG00000003604" "ENSMUSG00000003617" "ENSMUSG00000003623"
 [361] "ENSMUSG00000003644" "ENSMUSG00000003660" "ENSMUSG00000003665"
 [364] "ENSMUSG00000003731" "ENSMUSG00000003746" "ENSMUSG00000003752"
 [367] "ENSMUSG00000003779" "ENSMUSG00000003809" "ENSMUSG00000003812"
 [370] "ENSMUSG00000003813" "ENSMUSG00000003814" "ENSMUSG00000003824"
 [373] "ENSMUSG00000003848" "ENSMUSG00000003849" "ENSMUSG00000003863"
 [376] "ENSMUSG00000003865" "ENSMUSG00000003872" "ENSMUSG00000003873"
 [379] "ENSMUSG00000003882" "ENSMUSG00000003923" "ENSMUSG00000003934"
 [382] "ENSMUSG00000003970" "ENSMUSG00000003974" "ENSMUSG00000004018"
 [385] "ENSMUSG00000004032" "ENSMUSG00000004035" "ENSMUSG00000004038"
 [388] "ENSMUSG00000004040" "ENSMUSG00000004043" "ENSMUSG00000004044"
 [391] "ENSMUSG00000004054" "ENSMUSG00000004056" "ENSMUSG00000004070"
 [394] "ENSMUSG00000

 [703] "ENSMUSG00000008373" "ENSMUSG00000008435" "ENSMUSG00000008438"
 [706] "ENSMUSG00000008461" "ENSMUSG00000008475" "ENSMUSG00000008496"
 [709] "ENSMUSG00000008540" "ENSMUSG00000008590" "ENSMUSG00000008601"
 [712] "ENSMUSG00000008658" "ENSMUSG00000008668" "ENSMUSG00000008682"
 [715] "ENSMUSG00000008683" "ENSMUSG00000008690" "ENSMUSG00000008730"
 [718] "ENSMUSG00000008763" "ENSMUSG00000008845" "ENSMUSG00000008855"
 [721] "ENSMUSG00000008859" "ENSMUSG00000008892" "ENSMUSG00000008932"
 [724] "ENSMUSG00000008976" "ENSMUSG00000008999" "ENSMUSG00000009013"
 [727] "ENSMUSG00000009030" "ENSMUSG00000009073" "ENSMUSG00000009076"
 [730] "ENSMUSG00000009090" "ENSMUSG00000009092" "ENSMUSG00000009108"
 [733] "ENSMUSG00000009246" "ENSMUSG00000009281" "ENSMUSG00000009292"
 [736] "ENSMUSG00000009293" "ENSMUSG00000009350" "ENSMUSG00000009356"
 [739] "ENSMUSG00000009376" "ENSMUSG00000009394" "ENSMUSG00000009406"
 [742] "ENSMUSG00000009470" "ENSMUSG00000009487" "ENSMUSG00000009535"
 [745] "ENSMUSG00000

[1054] "ENSMUSG00000017718" "ENSMUSG00000017721" "ENSMUSG00000017723"
[1057] "ENSMUSG00000017724" "ENSMUSG00000017733" "ENSMUSG00000017737"
[1060] "ENSMUSG00000017740" "ENSMUSG00000017747" "ENSMUSG00000017754"
[1063] "ENSMUSG00000017756" "ENSMUSG00000017760" "ENSMUSG00000017765"
[1066] "ENSMUSG00000017774" "ENSMUSG00000017776" "ENSMUSG00000017778"
[1069] "ENSMUSG00000017781" "ENSMUSG00000017831" "ENSMUSG00000017837"
[1072] "ENSMUSG00000017843" "ENSMUSG00000017858" "ENSMUSG00000017861"
[1075] "ENSMUSG00000017868" "ENSMUSG00000017897" "ENSMUSG00000017929"
[1078] "ENSMUSG00000017950" "ENSMUSG00000017969" "ENSMUSG00000018001"
[1081] "ENSMUSG00000018008" "ENSMUSG00000018012" "ENSMUSG00000018040"
[1084] "ENSMUSG00000018042" "ENSMUSG00000018068" "ENSMUSG00000018102"
[1087] "ENSMUSG00000018126" "ENSMUSG00000018143" "ENSMUSG00000018160"
[1090] "ENSMUSG00000018166" "ENSMUSG00000018167" "ENSMUSG00000018169"
[1093] "ENSMUSG00000018189" "ENSMUSG00000018209" "ENSMUSG00000018263"
[1096] "ENSMUSG00000

[1405] "ENSMUSG00000020308" "ENSMUSG00000020309" "ENSMUSG00000020310"
[1408] "ENSMUSG00000020311" "ENSMUSG00000020312" "ENSMUSG00000020315"
[1411] "ENSMUSG00000020321" "ENSMUSG00000020325" "ENSMUSG00000020326"
[1414] "ENSMUSG00000020327" "ENSMUSG00000020329" "ENSMUSG00000020330"
[1417] "ENSMUSG00000020331" "ENSMUSG00000020333" "ENSMUSG00000020334"
[1420] "ENSMUSG00000020335" "ENSMUSG00000020340" "ENSMUSG00000020346"
[1423] "ENSMUSG00000020349" "ENSMUSG00000020357" "ENSMUSG00000020359"
[1426] "ENSMUSG00000020361" "ENSMUSG00000020362" "ENSMUSG00000020363"
[1429] "ENSMUSG00000020364" "ENSMUSG00000020366" "ENSMUSG00000020368"
[1432] "ENSMUSG00000020372" "ENSMUSG00000020375" "ENSMUSG00000020376"
[1435] "ENSMUSG00000020377" "ENSMUSG00000020380" "ENSMUSG00000020383"
[1438] "ENSMUSG00000020386" "ENSMUSG00000020387" "ENSMUSG00000020390"
[1441] "ENSMUSG00000020393" "ENSMUSG00000020395" "ENSMUSG00000020397"
[1444] "ENSMUSG00000020399" "ENSMUSG00000020400" "ENSMUSG00000020402"
[1447] "ENSMUSG00000

[1756] "ENSMUSG00000021303" "ENSMUSG00000021311" "ENSMUSG00000021313"
[1759] "ENSMUSG00000021314" "ENSMUSG00000021318" "ENSMUSG00000021326"
[1762] "ENSMUSG00000021327" "ENSMUSG00000021335" "ENSMUSG00000021338"
[1765] "ENSMUSG00000021339" "ENSMUSG00000021340" "ENSMUSG00000021357"
[1768] "ENSMUSG00000021359" "ENSMUSG00000021364" "ENSMUSG00000021367"
[1771] "ENSMUSG00000021368" "ENSMUSG00000021374" "ENSMUSG00000021376"
[1774] "ENSMUSG00000021377" "ENSMUSG00000021379" "ENSMUSG00000021385"
[1777] "ENSMUSG00000021390" "ENSMUSG00000021391" "ENSMUSG00000021408"
[1780] "ENSMUSG00000021413" "ENSMUSG00000021417" "ENSMUSG00000021418"
[1783] "ENSMUSG00000021423" "ENSMUSG00000021428" "ENSMUSG00000021431"
[1786] "ENSMUSG00000021432" "ENSMUSG00000021448" "ENSMUSG00000021451"
[1789] "ENSMUSG00000021456" "ENSMUSG00000021457" "ENSMUSG00000021458"
[1792] "ENSMUSG00000021460" "ENSMUSG00000021461" "ENSMUSG00000021466"
[1795] "ENSMUSG00000021468" "ENSMUSG00000021469" "ENSMUSG00000021476"
[1798] "ENSMUSG00000

[2107] "ENSMUSG00000022436" "ENSMUSG00000022437" "ENSMUSG00000022438"
[2110] "ENSMUSG00000022442" "ENSMUSG00000022443" "ENSMUSG00000022449"
[2113] "ENSMUSG00000022450" "ENSMUSG00000022451" "ENSMUSG00000022452"
[2116] "ENSMUSG00000022454" "ENSMUSG00000022462" "ENSMUSG00000022463"
[2119] "ENSMUSG00000022464" "ENSMUSG00000022469" "ENSMUSG00000022471"
[2122] "ENSMUSG00000022474" "ENSMUSG00000022475" "ENSMUSG00000022476"
[2125] "ENSMUSG00000022477" "ENSMUSG00000022479" "ENSMUSG00000022483"
[2128] "ENSMUSG00000022488" "ENSMUSG00000022489" "ENSMUSG00000022496"
[2131] "ENSMUSG00000022501" "ENSMUSG00000022510" "ENSMUSG00000022512"
[2134] "ENSMUSG00000022514" "ENSMUSG00000022517" "ENSMUSG00000022525"
[2137] "ENSMUSG00000022528" "ENSMUSG00000022529" "ENSMUSG00000022534"
[2140] "ENSMUSG00000022544" "ENSMUSG00000022545" "ENSMUSG00000022546"
[2143] "ENSMUSG00000022548" "ENSMUSG00000022551" "ENSMUSG00000022553"
[2146] "ENSMUSG00000022555" "ENSMUSG00000022556" "ENSMUSG00000022557"
[2149] "ENSMUSG00000

[2458] "ENSMUSG00000024181" "ENSMUSG00000024182" "ENSMUSG00000024186"
[2461] "ENSMUSG00000024190" "ENSMUSG00000024191" "ENSMUSG00000024193"
[2464] "ENSMUSG00000024197" "ENSMUSG00000024201" "ENSMUSG00000024207"
[2467] "ENSMUSG00000024210" "ENSMUSG00000024211" "ENSMUSG00000024212"
[2470] "ENSMUSG00000024213" "ENSMUSG00000024217" "ENSMUSG00000024218"
[2473] "ENSMUSG00000024222" "ENSMUSG00000024225" "ENSMUSG00000024228"
[2476] "ENSMUSG00000024231" "ENSMUSG00000024232" "ENSMUSG00000024235"
[2479] "ENSMUSG00000024240" "ENSMUSG00000024241" "ENSMUSG00000024248"
[2482] "ENSMUSG00000024253" "ENSMUSG00000024254" "ENSMUSG00000024256"
[2485] "ENSMUSG00000024258" "ENSMUSG00000024270" "ENSMUSG00000024283"
[2488] "ENSMUSG00000024287" "ENSMUSG00000024290" "ENSMUSG00000024292"
[2491] "ENSMUSG00000024293" "ENSMUSG00000024298" "ENSMUSG00000024299"
[2494] "ENSMUSG00000024301" "ENSMUSG00000024304" "ENSMUSG00000024308"
[2497] "ENSMUSG00000024312" "ENSMUSG00000024317" "ENSMUSG00000024319"
[2500] "ENSMUSG00000

In [38]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids030_60.Rds",sep=""))

In [39]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)


ENSMUSG00000018442      62112      47945      64253      57390      64986
ENSMUSG00000018446      28704      23014      39095      24972      39659
ENSMUSG00000018459      14410       8211       6839       7081       7600
ENSMUSG00000018470        649        199        800        214        500
ENSMUSG00000018474     133592      53294     102071      93930      99644
ENSMUSG00000018476      22211       9105       8300       9661       8779
ENSMUSG00000018486        400        550        549        298        746
ENSMUSG00000018500          0         50          0          0          0
ENSMUSG00000018507        806        283        750        124        340
ENSMUSG00000018537       3517       1373       3851       1015       3834
ENSMUSG00000018541      10172       5556       7678       4712       6937
ENSMUSG00000018547      28158      10507      13423      10285      14842
ENSMUSG00000018548       8035       7041       7019       6360       7216
ENSMUSG00000018559      21169       8

ENSMUSG00000019715      25468      15830       9434      12354       9543
ENSMUSG00000019738      10826       8771       9988       4590       8383
ENSMUSG00000019761       1286        549       1014        782        886
ENSMUSG00000019762      70711      61437      51326      57748      57389
ENSMUSG00000019768      21250      10739      20502      39096      22144
ENSMUSG00000019772         50          0          0          0          0
ENSMUSG00000019773        550        288        250        600        200
ENSMUSG00000019774       1742       1395       3138       1662       2318
ENSMUSG00000019775         50          0        100          0        150
ENSMUSG00000019779       2438       3581       3291       3456       3326
ENSMUSG00000019782       3952       3442       2927       3446       2873
ENSMUSG00000019785          0          0          0          0          0
ENSMUSG00000019787        654          0          0          0          0
ENSMUSG00000019789        150         

ENSMUSG00000000817          0        150        150          0        142
ENSMUSG00000000823      26598      22445      19157      11449      32025
ENSMUSG00000000826      43722      38277      39290      30074      42702
ENSMUSG00000000869        150        100        150        106        196
ENSMUSG00000000876     160388      34879      95274      33030     126797
ENSMUSG00000000881       9925      11711      12100      10348      12881
ENSMUSG00000000889          0          0          0          0          0
ENSMUSG00000000901       2525      36815      43320      20853      55977
ENSMUSG00000000902       9104       8851      12085       4573      21646
ENSMUSG00000000903          0          0        100        350         99
ENSMUSG00000000915      24032      23061      33233       6158      28960
ENSMUSG00000000942          0          0          0         50        299
ENSMUSG00000000957      17594      10474      21183       6550      39323
ENSMUSG00000000958       2448       22

ENSMUSG00000001985          0          0         50          0          0
ENSMUSG00000001986        700        893        442        350        897
ENSMUSG00000001998       4873       4380       4283       5199       6164
ENSMUSG00000001999       7572       6530       4946       6014       8441
ENSMUSG00000002010      25923      45067      47498      22788      39689
ENSMUSG00000002015      81616      70916      62332      58662     102229
ENSMUSG00000002020       1047        132       1366        531       1379
ENSMUSG00000002028      18786      13562      17919      11746      19033
ENSMUSG00000002031      15410      15114      12795      12450      21508
ENSMUSG00000002033        548        150        637        200        490
ENSMUSG00000002043       6440       4947       4748       2578      12713
ENSMUSG00000002052      37762      29536      31638      17865      48264
ENSMUSG00000002059       1527       1778       1985       1082       2120
ENSMUSG00000002068        386       18

ENSMUSG00000003131      22277      17847      15960      12666      21823
ENSMUSG00000003135      10033       9472      10118       8029      11734
ENSMUSG00000003153        599        200        399         98        295
ENSMUSG00000003161      13927      19715      14433      12730      18546
ENSMUSG00000003184      32920      35733      35819      18113      43567
ENSMUSG00000003200       8186       9617      12034       5370      11477
ENSMUSG00000003206        747          0        599         50        299
ENSMUSG00000003208       1238       2474       2191       1196       3299
ENSMUSG00000003226      46587      45403      48991      56895      42940
ENSMUSG00000003227          0          0          0          0          0
ENSMUSG00000003228       5965        995      13612        850       7716
ENSMUSG00000003233      11092      11317      12025       5856      16977
ENSMUSG00000003235      20615      22298      21605      13257      22820
ENSMUSG00000003269       7853      119

...

ENSMUSG00000006705      11019      14637       2587      11516      10848
ENSMUSG00000006711       1385        476        514        522        587
ENSMUSG00000006715      14527      21361      10321      21149      23783
ENSMUSG00000006717      20796      11634       6699      15048      12029
ENSMUSG00000006720        300        500        150        650        617
ENSMUSG00000006724          0        100          0         50         16
ENSMUSG00000006728      83561      77750      46403      89614      62752
ENSMUSG00000006731      21942      39826      12071      37605      37387
ENSMUSG00000006740      38562      48087      15167      47853      53684
ENSMUSG00000006764          0          0          0          0          0
ENSMUSG00000006777       1277       1377       1124       1632       1108
ENSMUSG00000006818     214104     196112      55558     193396     171101
ENSMUSG00000006906      10337      15420       3545      15630      15684
ENSMUSG00000006932     180571     2465

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



      60426      37404
ENSMUSG00000015085      53542     105184      17641      72653     119995
ENSMUSG00000015090         80        150         50          0        191
ENSMUSG00000015094      32772      61719       8689      46842      64609
ENSMUSG00000015095      48368      57281      14465      65251      74977
ENSMUSG00000015112      90365      50535      26667      66369      57199
ENSMUSG00000015120      13330      15829       8427      25905      23339
ENSMUSG00000015127      16275      15359       6186      17835      11498
ENSMUSG00000015134         50        600       1291        626        567
ENSMUSG00000015143      51517      47183      15969      50751      27382
ENSMUSG00000015149      33058      42323      11524      48064      51305
ENSMUSG00000015165      49200      64403      32512      93135      70361
ENSMUSG00000015214      30429      36896      11479      32134      32254
ENSMUSG00000015243      84645     117073      61998     151164     139652
ENSMUSG00000015

ENSMUSG00000016619      27299      35131      10334      36660      27501
ENSMUSG00000016637       2419       2785        734       3607       3887
ENSMUSG00000016664      55033      51300      14980      66477      44557
ENSMUSG00000016757      19311      19121       3667      16656      14055
ENSMUSG00000016763       1299       2642        693       4511       1533
ENSMUSG00000016833       7843       9170       5586       8491       8842
ENSMUSG00000016933      54669      53146      17648      51187      61796
ENSMUSG00000016942     165782     141414      89405     175617     147835
ENSMUSG00000016995        100          0          0          0         50
ENSMUSG00000017002       4398       2823       2116       4914       4979
ENSMUSG00000017009      60437     104436      93964     180289     124152
ENSMUSG00000017057      18435      22259      14862      21092      20169
ENSMUSG00000017119     156590     182699      60753     210855     169150
ENSMUSG00000017132      16531      218

ENSMUSG00000018433      11613      15890       3471      11801      13207
ENSMUSG00000018442      34707      51330      26979      45689      48961
ENSMUSG00000018446     103684     127724      21742      82998     119097
ENSMUSG00000018459     114963     206539      42854     159152     262700
ENSMUSG00000018470        100        342         50        409        422
ENSMUSG00000018474      65975      50420      18350      62377      73362
ENSMUSG00000018476      12678      14715       6804      41967      19361
ENSMUSG00000018486       2400       4761        350       3878       4489
ENSMUSG00000018500        192        399        100        200        499
ENSMUSG00000018507       4384       7755       1286       9762       9828
ENSMUSG00000018537       2938       5358       1623       7434       5097
ENSMUSG00000018541       6596       9926       3217      11029      10605
ENSMUSG00000018547      12676      13564       9007      23501      11136
ENSMUSG00000018548      19487      284

In [40]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df030_time_course_60.Rds",sep=""))

In [41]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data030_time_course_60.Rds", sep=""))